DUMMY DATA

In [18]:
import torch
import torch.nn as nn

batch_size = 5
feature_dim = 37
length_time = 50

data = torch.randn(batch_size, feature_dim, length_time)
time = torch.randint(0, 500, (1, length_time))
data_1 = data[0]


static = torch.randn(batch_size, 8)
static_1 = static[0]
    
sensor_mask = torch.zeros(batch_size, feature_dim, length_time)

sensor_mask[0, :10] = 1
sensor_mask[1, :20] = 1

SENSOR EMBEDDING LAYER

In [23]:


class SensorEmbeddingLayer(nn.Module):
    """Embedding layer for sensor features."""

    def __init__(self, num_sensor: int, time_length: int, dim_embedding: int):
        super(SensorEmbeddingLayer, self).__init__()
        self.num_sensor = num_sensor
        self.embedding_size = time_length
        self.dim_embedding = dim_embedding

        # Define the embedding layer
        self.sensor_embedding = nn.Linear(time_length, dim_embedding)

    def forward(self, sensor_matrix: torch.Tensor) -> torch.Tensor:
        """
        Apply sensor embedding to the input sensor matrix.
        
        Parameters:
        sensor_matrix (torch.Tensor): Input tensor of shape (num_sensor, embedding_size)
        
        Returns:
        torch.Tensor: Output tensor of shape (num_sensor, embedding_size, dim_embedding)
        """
        # Apply the embedding layer to each sensor in the matrix
        embedded_matrix = self.sensor_embedding(sensor_matrix)
        
        # Expand the dimensions to match the desired output shape
        embedded_matrix = embedded_matrix.unsqueeze(1).expand(-1, self.embedding_size, -1)
        
        return embedded_matrix

# Example usage
num_sensor = 37
embedding_size = length_time
dim_embedding = 32

# Create the sensor embedding layer
sensor_embedding_layer = SensorEmbeddingLayer(num_sensor, embedding_size, dim_embedding)


# Get the embedded output
sensor_embedded_output = sensor_embedding_layer(data_1)

print("Embedded Output Shape:", sensor_embedded_output.shape)
print("Embedded Output:", sensor_embedded_output)

Embedded Output Shape: torch.Size([37, 50, 32])
Embedded Output: tensor([[[-0.7872, -1.2312,  0.8621,  ..., -0.4186,  0.1538,  0.1275],
         [-0.7872, -1.2312,  0.8621,  ..., -0.4186,  0.1538,  0.1275],
         [-0.7872, -1.2312,  0.8621,  ..., -0.4186,  0.1538,  0.1275],
         ...,
         [-0.7872, -1.2312,  0.8621,  ..., -0.4186,  0.1538,  0.1275],
         [-0.7872, -1.2312,  0.8621,  ..., -0.4186,  0.1538,  0.1275],
         [-0.7872, -1.2312,  0.8621,  ..., -0.4186,  0.1538,  0.1275]],

        [[-0.5446,  0.7237,  0.0139,  ..., -0.1148, -0.1133,  0.2608],
         [-0.5446,  0.7237,  0.0139,  ..., -0.1148, -0.1133,  0.2608],
         [-0.5446,  0.7237,  0.0139,  ..., -0.1148, -0.1133,  0.2608],
         ...,
         [-0.5446,  0.7237,  0.0139,  ..., -0.1148, -0.1133,  0.2608],
         [-0.5446,  0.7237,  0.0139,  ..., -0.1148, -0.1133,  0.2608],
         [-0.5446,  0.7237,  0.0139,  ..., -0.1148, -0.1133,  0.2608]],

        [[-0.0079, -0.5118,  0.7526,  ..., -0.5490,

TIME EMBEDDING LAYER

In [20]:
import math
from typing import Any, Optional

import torch
from torch import nn
from transformers import BigBirdConfig, MambaConfig

class TimeEmbeddingLayer(nn.Module):
    """Embedding layer for time features."""

    def __init__(self, embedding_size: int, is_time_delta: bool = False):
        super().__init__()
        self.embedding_size = embedding_size
        self.is_time_delta = is_time_delta

        self.w = nn.Parameter(torch.empty(1, self.embedding_size))
        self.phi = nn.Parameter(torch.empty(1, self.embedding_size))

        nn.init.xavier_uniform_(self.w)
        nn.init.xavier_uniform_(self.phi)

    def forward(self, time_stamps: torch.Tensor) -> Any:
        """Apply time embedding to the input time stamps."""
        if self.is_time_delta:
            # If the time_stamps represent time deltas, we calculate the deltas.
            # This is equivalent to the difference between consecutive elements.
            time_stamps = torch.cat(
                (time_stamps[:, 0:1] * 0, time_stamps[:, 1:] - time_stamps[:, :-1]),
                dim=-1,
            )
        time_stamps = time_stamps.float()
        time_stamps_expanded = time_stamps.unsqueeze(-1)
        next_input = time_stamps_expanded * self.w + self.phi

        return torch.sin(next_input)
    
# Example usage

time_layer = TimeEmbeddingLayer(dim_embedding, is_time_delta=False)

# Get the embedded output
time_embedded_output = time_layer(time)

print("Time Embedded Output Shape:", time_embedded_output.shape)

Time Embedded Output Shape: torch.Size([1, 50, 32])


STATIC EMBEDDING

In [26]:
class StaticEmbeddings(nn.Module):
    """Embedding layer for static features."""

    def __init__(self, input_dim: int, embedding_dim: int):
        super(StaticEmbeddings, self).__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim

        # Define the embedding layer
        self.embedding_layer = nn.Linear(input_dim, embedding_dim)

    def forward(self, static_features: torch.Tensor) -> torch.Tensor:
        """
        Apply embedding to the input static features.
        
        Parameters:
        static_features (torch.Tensor): Input tensor of shape (batch_size, input_dim)
        
        Returns:
        torch.Tensor: Output tensor of shape (batch_size, embedding_dim)
        """
        # Apply the embedding layer
        embedded_features = self.embedding_layer(static_features)
        
        return embedded_features

# Example usage
input_dim = static.shape[1]
embedding_dim = 32  # Ensure this matches the desired embedding dimension

# Create the static embedding layer
static_embedding_layer = StaticEmbeddings(input_dim, embedding_dim)

# Get the embedded output
static_embedded_output = static_embedding_layer(static_1)

print("Static Embedded Output Shape:", static_embedded_output.shape)
print("Static Embedded Output:", static_embedded_output)

Static Embedded Output Shape: torch.Size([32])
Static Embedded Output: tensor([-0.3539, -0.0950, -0.2146, -0.1592, -0.6021, -0.5876,  0.1616,  0.3047,
        -0.2513,  0.3401,  0.4000,  0.1131,  0.1653, -0.0472, -1.0387,  0.3853,
        -0.1978,  0.1301, -0.0531,  0.6286,  0.3585, -0.0314, -0.3951,  0.8722,
         0.2968, -0.0645,  0.2482, -0.4026, -0.4583, -0.4860, -0.2026,  0.3073],
       grad_fn=<ViewBackward0>)


TOTAL EMBEDDING

In [ ]:
class CombinedEmbeddings(nn.Module):
    """Combined embedding layer for sensor, time, and static features."""

    def __init__(self, num_sensor: int, time_lenght: int, dim_embedding: int, static_input_dim: int, is_time_delta: bool = False):
        super(CombinedEmbeddings, self).__init__()
        
        # Initialize sensor embedding layer
        self.sensor_embedding_layer = SensorEmbeddingLayer(num_sensor, time_lenght, dim_embedding)
        
        # Initialize time embedding layer
        self.time_embedding_layer = TimeEmbeddingLayer(dim_embedding, is_time_delta)
        
        # Initialize static embedding layer
        self.static_embedding_layer = StaticEmbeddings(static_input_dim, dim_embedding)

    def forward(self, sensor_matrix: torch.Tensor, time_stamps: torch.Tensor, static_features: torch.Tensor) -> torch.Tensor:
        """
        Apply combined embeddings to the input sensor matrix, time stamps, and static features.
        
        Parameters:
        sensor_matrix (torch.Tensor): Input tensor of shape (num_sensor, sensor_embedding_size)
        time_stamps (torch.Tensor): Input tensor of shape (1, time_embedding_size)
        static_features (torch.Tensor): Input tensor of shape (batch_size, static_input_dim)
        
        Returns:
        torch.Tensor: Concatenated output tensor of shape (batch_size, combined_embedding_dim)
        """
        # Apply sensor embedding
        sensor_embedded = self.sensor_embedding_layer(sensor_matrix) # output shape (num_sensor, time_lenght, dim_embedding)
        
        # Apply time embedding
        time_embedded = self.time_embedding_layer(time_stamps) # output shape (1, time_lenght, dim_embedding)
        time_embedded = time_embedded.expand(sensor_embedded.shape) # transform to shape (num_sensor, time_lenght, dim_embedding)
        
        # Apply static embedding
        static_embedded = self.static_embedding_layer(static_features) # output shape (dim_embedding)
        static_embedded = static_embedded.unsqueeze(0).unsqueeze(0).expand(sensor_embedded.shape) # transform to shape (num_sensor, time_lenght, dim_embedding)
        
        # Combine all embeddings by adding them together
        combined_embedding = sensor_embedded + time_embedded + static_embedded
        
        return combined_embedding

# Example usage
combined_embedding_layer = CombinedEmbeddings(num_sensor, embedding_size, dim_embedding, input_dim, is_time_delta=False) 

# Get the combined embedded output
combined_embedded_output = combined_embedding_layer(data_1, time, static_1)

print("Combined Embedded Output Shape:", combined_embedded_output.shape)
print("Combined Embedded Output:", combined_embedded_output)

Combined Embedded Output Shape: torch.Size([37, 50, 32])
Combined Embedded Output: tensor([[[ 1.4674e-01, -5.3012e-01, -2.0223e+00,  ..., -6.0152e-01,
           1.4407e+00, -1.7751e+00],
         [ 5.3187e-01, -1.7666e+00, -5.4897e-01,  ...,  2.5357e-01,
           1.6419e+00, -9.6679e-01],
         [ 7.5946e-01, -1.2530e+00, -2.2520e+00,  ...,  6.7409e-01,
           4.9242e-01, -1.6938e+00],
         ...,
         [-8.5112e-01, -3.2267e-01, -2.0749e+00,  ..., -6.5844e-01,
           9.8222e-01, -7.2803e-01],
         [ 2.8971e-01, -1.8018e+00, -3.1125e-01,  ...,  3.7465e-02,
          -9.1286e-03, -2.0416e-01],
         [ 4.9728e-01, -5.7990e-01, -1.9952e+00,  ...,  3.1725e-01,
           5.7610e-01, -1.7324e+00]],

        [[-4.0913e-01,  1.3280e-01, -1.2357e+00,  ..., -8.7793e-01,
           1.5883e+00, -1.2788e+00],
         [-2.3999e-02, -1.1037e+00,  2.3760e-01,  ..., -2.2842e-02,
           1.7895e+00, -4.7042e-01],
         [ 2.0359e-01, -5.9005e-01, -1.4654e+00,  ...,  3.976

MAMBA MODEL